In [2]:
pip install -q langchain 


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [4]:
from langchain.chat_models import init_chat_model

In [4]:
pip install -U langchain-groq


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# llm = init_chat_model("google_genai:gemini-2.0-flash")
llm = init_chat_model("groq:llama3-70b-8192")
# llm.invoke("who was the first person to walk on the moon")

class State(TypedDict):
    messages: Annotated[list, add_messages]# reduceer function 

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [6]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
# message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages":[message]})
 
response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='5731a933-42d0-4c90-b1b4-9be9e53417e1'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 24, 'total_tokens': 27, 'completion_time': 2.3e-07, 'prompt_time': 0.010890506, 'queue_time': 0.279702592, 'total_time': 0.010890736}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_bf16903a67', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--78048906-c175-4587-8889-5240b2f3cce2-0', usage_metadata={'input_tokens': 24, 'output_tokens': 3, 'total_tokens': 27})]

In [ ]:
state = None
while True:
    in_message = "who was the presiden to india"
    if in_message.lower() in {"quit","exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

In [9]:
state = None
while True:
    in_message = input("You: ")  # <-- Ask user for input dynamically
    if in_message.lower() in {"quit", "exit"}:
        break

    if state is None:
        state = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    # Invoke the LLM (graph)
    state = graph.invoke(state)

    # Print the last bot message
    print("Bot:", state["messages"][-1].content)


Bot: Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?
Bot: The first Prime Minister of India was Jawaharlal Nehru. He served as the Prime Minister of India from August 15, 1947, until May 27, 1964.
